In [2]:
import torch
import torchvision.models as models
from torch import nn
import os

device = "cuda" if torch.cuda.is_available() else "cpu"
NUM_WORKERS = os.cpu_count()

In [3]:
output_shape = 2
model = models.mobilenet_v3_large() 

model.classifier = nn.Sequential(
    nn.Linear(in_features=960, out_features=1024, bias=True),
    nn.Hardswish(),
    nn.Dropout(p=0.4),
    nn.Linear(in_features=1024, out_features=output_shape, bias=True),
).to(device)

checkpoint = torch.load('models/model_6_mobilenet.pth', map_location=device)
model_state_dict = checkpoint['model_state_dict']

model.load_state_dict(model_state_dict)
model.eval()

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi

In [4]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay = 1e-4)
loss_fn = torch.nn.CrossEntropyLoss()

optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [5]:
model.to(device)

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi

In [26]:
from torchvision import datasets
from torchvision.transforms import v2
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

val_transform = v2.Compose([
    v2.CenterCrop(384),
    v2.PILToTensor(),
    v2.ToDtype(torch.float32, scale=True),
])

class ImageFolderWithFilenames(ImageFolder):
    def __getitem__(self, index):
        path, target = self.samples[index]
        image = self.loader(path)
        
        # Get the filename from the path
        filename = path.split('/')[-1]  # Modify this according to your file path structure
        
        if self.transform is not None:
            image = self.transform(image)
        
        if self.target_transform is not None:
            target = self.target_transform(target)
        
        return image, target, filename

test_data = ImageFolderWithFilenames('final_dataset/test', transform = val_transform)
class_names = test_data.classes

testloader = DataLoader(test_data,
                        batch_size=32,
                        shuffle=False,
                        num_workers=2,
                        pin_memory=True)

testloader, class_names


(<torch.utils.data.dataloader.DataLoader at 0x7f432b3ea410>, ['0', '1'])

In [28]:
import torch.nn.functional as F
import csv

y_pred = []
y_true = []
filenames = []

# iterate over test data
with torch.no_grad():
    for input, label, filename in testloader:
        input = input.to(device)
        output = model(input) # Feed Network
        probabilities = F.softmax(output, dim=1)
        _, predicted_classes = torch.max(probabilities, 1)

        y_pred.extend(predicted_classes) # Save Prediction

        label = label.data.cpu().numpy()
        y_true.extend(label) # Save Truth

        filenames.extend(filename)  # Save File Names

y_pred = [pred.item() for pred in y_pred]
print("y_pred =", y_pred)
print("y_true =", y_true)

# Save to CSV file
csv_data = list(zip(filenames, y_true, y_pred))
csv_file_path = 'models/mobilenet_predictions.csv'

with open(csv_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['File Name', 'True Label', 'Predicted Label'])
    csv_writer.writerows(csv_data)

print(f"Predictions saved to {csv_file_path}")

/tmp/ipykernel_10246/779287068.py:1: DeprecationWarning: 'cgi' is deprecated and slated for removal in Python 3.13
  from cgi import test


y_pred = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 

In [30]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix

# Assuming y_true and y_pred are NumPy arrays or Python lists
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Build confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

Accuracy: 83.01%
Confusion Matrix:
[[1776   29]
 [ 593 1264]]
